In [1]:
#setup imports and make ure the files we care about exist
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os.path

#custom modules
import feature_extraction
import unsw_nb15_dataset
import generator_discriminator
import train

from importlib import reload #allow us to reload custom modules any time we like with modifications
reload(feature_extraction)
reload(unsw_nb15_dataset)
reload(generator_discriminator)
reload(train)

#fe.test_cases()

<module 'train' from '/home/jaywalker/MachineLearning/PacketGAN/train.py'>

In [ ]:
#set some hyperparameters
sequence_length=25
batch_size=1

#first, load the dataset
data_set = unsw_nb15_dataset.UNSW_NB15(['/home/jaywalker/MachineLearning/PacketGAN/UNSW-NB15_1_clean.csv'],
                                       sequence_length=sequence_length, 
                                       transform=feature_extraction.build_feature_sequence_tensor)

#pick out the attack examples
data_set.use_only_category('DoS'); #in this case just look at DoS attack samples

#convert attack examples into time windows for training
#DONE by the UNSW_NB15 class at load-time

#convert each window into appropriate feature tensor
#DONE by the transform function passed to the UNSW_NB15 class at load-time

#train GAN to replace masked packets
#by iterating over the dataset for some # of epochs
#TODO make a python file to contain the network class definitions and training funciton

data_loader = DataLoader(data_set, batch_size=batch_size,
                        shuffle=True, num_workers=4)

data_example = None
for count, data in enumerate(data_loader):
    data_example = data
    if count == 0:
        break
        
G = generator_discriminator.Generator(sequence_length, data_example.shape[2], data_example.shape[2] * 2, 30, data_example.shape[2], batch_size, torch.sigmoid)
D = generator_discriminator.Discriminator(sequence_length, data_example.shape[2], data_example.shape[2] * 2, 1, batch_size, torch.sigmoid)

In [ ]:
#Train the network!
gen, g_losses, df_losses, g_stats, df_stats = train.train(G,D,data_loader,25)

#plot instantaneous losses
plt.plot(range(len(df_losses)), df_losses, 'r-', range(len(g_losses)), g_losses, 'g-')
plt.show()

#plot loss averages
plt.plot(range(len(df_stats.get_averages())), df_stats.get_averages(), 'r-', range(len(g_stats.get_averages())), g_stats.get_averages(), 'g-')
plt.show()

Training has started...
Epoch:  0
D Real Error:  0.5489287376403809
D Fake Error:  0.7951973080635071
G Error:  0.6931566596031189
Remaining time: 3:59:47.709969


In [ ]:
import random
data_example = None
for count, data in enumerate(data_loader):
    data_example = data
    if count == 0:
        break
print("Real data: ", feature_extraction.decode_feature_sequence_tensor(data_set, data_example[0]), end="\n\n")


#note here that "gen" is the trained generator returned from the training function above
generated_data = G(data_example).detach()
#print("Shape of generated data: ", generated_data.shape)
#torch.set_printoptions(profile="full")
#print("Generated data: ", generated_data[0])
print("Fake data: ", feature_extraction.decode_feature_sequence_tensor(data_set, G(data_example).detach()[0]))

#print("From data set:")
#print(feature_extraction.decode_feature_sequence_tensor(data_set, data_set[0]))